In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import pyplot
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from collections import Counter
from sklearn.preprocessing import OneHotEncoder

In [70]:
filePath="/Users/a0p07yx/Documents/Projects/HACKATHONS/DATA/1LTFS/Train/train_data.xlsx"
df=pd.read_excel(filePath,header=0,skiprows=0,delimiter=",")
rows,columns=df.shape
df.head()

,ID,Frequency,InstlmentMode,LoanStatus,PaymentMode,BranchID,Area,Tenure,AssetCost,AmountFinance,...,ManufacturerID,SupplierID,LTV,SEX,AGE,MonthlyIncome,City,State,ZiPCODE,Top-up Month
0,1,Monthly,Arrear,Closed,PDC_E,1,NaN,48,450000,275000.0,...,1568,21946,61.11,M,49.0,35833.33,RAISEN,MADHYA PRADESH,464993.0,> 48 Months
1,2,Monthly,Advance,Closed,PDC,333,BHOPAL,47,485000,350000.0,...,1062,34802,70.00,M,23.0,666.67,SEHORE,MADHYA PRADESH,466001.0,No Top-up Service
2,3,Quatrly,Arrear,Active,Direct Debit,1,NaN,68,690000,519728.0,...,1060,127335,69.77,M,39.0,45257.00,BHOPAL,MADHYA PRADESH,462030.0,12-18 Months
3,7,Monthly,Advance,Closed,Billed,125,GUNA,48,480000,400000.0,...,1060,25094,80.92,M,24.0,20833.33,ASHOK NAGAR,MADHYA PRADESH,473335.0,> 48 Months
4,8,Monthly,Arrear,Closed,Billed,152,BILASPUR,44,619265,440000.0,...,1046,21853,71.05,M,56.0,27313.67,BILASPUR,CHATTISGARH,495442.0,36-48 Months


In [103]:
filePath="/Users/a0p07yx/Documents/Projects/HACKATHONS/DATA/1LTFS/Train/train_bureau.xlsx"
df3=pd.read_excel(filePath,header=0,skiprows=0,delimiter=",")
rows,columns=df3.shape
df3.head()

,ID,SELF-INDICATOR,MATCH-TYPE,ACCT-TYPE,CONTRIBUTOR-TYPE,DATE-REPORTED,OWNERSHIP-IND,ACCOUNT-STATUS,DISBURSED-DT,CLOSE-DT,...,INSTALLMENT-FREQUENCY,OVERDUE-AMT,WRITE-OFF-AMT,ASSET_CLASS,REPORTED DATE - HIST,DPD - HIST,CUR BAL - HIST,AMT OVERDUE - HIST,AMT PAID - HIST,TENURE
0,1,False,PRIMARY,Overdraft,NAB,2018-04-30,Individual,Delinquent,2015-10-05,NaN,...,NaN,"37,873",0.0,Standard,"20180430,20180331,",030000,"37873,12820,","37873,,",",,",NaN
1,1,False,PRIMARY,Auto Loan (Personal),NAB,2019-12-31,Individual,Active,2018-03-19,NaN,...,F03,NaN,0.0,Standard,"20191231,20191130,20191031,20190930,20190831,2...",0000000000000000000000000000000000000000000000...,"20797,21988,23174,24341,25504,26648,27780,2891...",",,,,,,,,,,,,,,,,,,,,1452,,",",,,,,,,,,,,,,,,,,,,,,,",36.0
2,1,True,PRIMARY,Tractor Loan,NBF,2020-01-31,Individual,Active,2019-08-30,NaN,...,NaN,0,0.0,NaN,"20200131,20191231,20191130,20191031,20190930,2...",000000000000000000,"116087,116087,145000,145000,145000,145000,","0,0,0,0,0,0,",",,,,,,",NaN
3,1,True,PRIMARY,Auto Loan (Personal),NBF,2017-09-30,Individual,Closed,2013-09-27,2017-09-21 00:00:00,...,NaN,0,0.0,NaN,"20170930,20170801,20170731,20170630,20170531,2...",000DDD0270260270260270240270270000320000000000...,"0,,15925,23754,31494,39147,46713,54194,61590,6...","0,,1014,1014,1014,1014,1014,1014,1014,983,0,92...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,",NaN
4,1,True,PRIMARY,Tractor Loan,NBF,2016-02-29,Individual,Closed,2012-02-10,2016-02-01 00:00:00,...,NaN,0,0.0,NaN,"20160229,20160131,20151231,20151130,20151031,2...",0000000000000000000000000000000000000000000000...,"0,0,23658,23321,22989,46321,45662,45012,68030,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,",NaN


In [94]:
def split_aggregate(row):   
    final_result=0
    x=[]
    if type(row)!=type(2.3):
            x=row.split(',')
    
    sumi=0
    count=0
    for val in x:
        count=count+1
        if len(val)!=0:
            sumi=sumi+int(val)
    if count==0:
        return 0.0
    else:
        return (sumi*1.0)/count

In [95]:
def installment_adj(row):
    if type(row)==type(2.3):
            return row
    if len(row)==0:
        return row
   
    install=row.split('/')
    return int(install[0].replace(',', ''))

In [96]:
def converToInt(row):
    if type(row)==type(2.3):
        return 0
    if type(row)==type(23):
        return row
    return int(row.replace(',',''))

In [104]:
df3=df3.drop(['CREDIT-LIMIT/SANC AMT','DATE-REPORTED','DPD - HIST','REPORTED DATE - HIST','DISBURSED-DT','CLOSE-DT','LAST-PAYMENT-DATE'],axis=1)

In [105]:
df3['CUR BAL - HIST'] = df3['CUR BAL - HIST'].apply(split_aggregate)

In [106]:
df3['AMT OVERDUE - HIST'] = df3['AMT OVERDUE - HIST'].apply(split_aggregate)

In [107]:
df3['AMT PAID - HIST'] = df3['AMT OVERDUE - HIST'].apply(split_aggregate)

In [108]:
df3['INSTALLMENT-AMT']=df3['INSTALLMENT-AMT'].apply(installment_adj)

In [109]:
df3['CURRENT-BAL']= df3['CURRENT-BAL'].apply(converToInt)

In [110]:
df3['DISBURSED-AMT/HIGH CREDIT']= df3['DISBURSED-AMT/HIGH CREDIT'].apply(converToInt)

In [111]:
df3['OVERDUE-AMT']= df3['OVERDUE-AMT'].apply(converToInt)

In [114]:
df3['WRITE-OFF-AMT']=df3['WRITE-OFF-AMT'].fillna(df3['WRITE-OFF-AMT'].mean())
df3['OVERDUE-AMT']=df3['OVERDUE-AMT'].fillna(df3['OVERDUE-AMT'].mean())
df3['TENURE']=df3['TENURE'].fillna(df3['TENURE'].mean())

df3['ASSET_CLASS']=df3['ASSET_CLASS'].fillna('Standard')
df3['CURRENT-BAL']=df3['CURRENT-BAL'].fillna(df3['CURRENT-BAL'].mean())
df3['INSTALLMENT-AMT']=df3['INSTALLMENT-AMT'].fillna(df3['INSTALLMENT-AMT'].mean())
df3['INSTALLMENT-FREQUENCY']=df3['INSTALLMENT-FREQUENCY'].fillna(df3['INSTALLMENT-FREQUENCY'].mode().iloc[0])

In [120]:
#df.isnull().sum()

ID                 0
Frequency          0
InstlmentMode      0
LoanStatus         0
PaymentMode        0
BranchID           0
Tenure             0
AssetCost          0
AmountFinance      0
DisbursalAmount    0
EMI                0
AssetID            0
ManufacturerID     0
SupplierID         0
LTV                0
SEX                0
AGE                0
MonthlyIncome      0
State              0
Top-up Month       0
dtype: int64

In [117]:
df=df.drop(['DisbursalDate','MaturityDAte','AuthDate','Area','City','ZiPCODE'],axis=1)

In [119]:
df['SEX']=df['SEX'].fillna(df['SEX'].mode().iloc[0])
df['AGE']=df['AGE'].fillna(df['AGE'].mean())
df['MonthlyIncome']=df['MonthlyIncome'].fillna(df['MonthlyIncome'].mean())

In [16]:
#df3.head()

,ID,SELF-INDICATOR,MATCH-TYPE,ACCT-TYPE,CONTRIBUTOR-TYPE,OWNERSHIP-IND,ACCOUNT-STATUS,DISBURSED-AMT/HIGH CREDIT,INSTALLMENT-AMT,CURRENT-BAL,INSTALLMENT-FREQUENCY,OVERDUE-AMT,WRITE-OFF-AMT,ASSET_CLASS,CUR BAL - HIST,AMT OVERDUE - HIST,AMT PAID - HIST,TENURE
0,1,False,PRIMARY,Overdraft,NAB,Individual,Delinquent,37352,60961.000948,37873,F03,37873,0.0,Standard,16897.666667,12624.333333,0.0,28.801029
1,1,False,PRIMARY,Auto Loan (Personal),NAB,Individual,Active,44000,1405.000000,20797,F03,0,0.0,Standard,31378.347826,63.130435,0.0,36.000000
2,1,True,PRIMARY,Tractor Loan,NBF,Individual,Active,145000,60961.000948,116087,F03,0,0.0,Standard,116024.857143,0.000000,0.0,28.801029
3,1,True,PRIMARY,Auto Loan (Personal),NBF,Individual,Closed,300000,60961.000948,0,F03,0,0.0,Standard,118875.972973,471.916667,0.0,28.801029
4,1,True,PRIMARY,Tractor Loan,NBF,Individual,Closed,275000,60961.000948,0,F03,0,0.0,Standard,114983.000000,0.000000,0.0,28.801029


In [46]:
#df.head()

,ID,Frequency,InstlmentMode,LoanStatus,PaymentMode,BranchID,Area,Tenure,AssetCost,AmountFinance,...,ManufacturerID,SupplierID,LTV,SEX,AGE,MonthlyIncome,City,State,ZiPCODE,Top-up Month
0,1,Monthly,Arrear,Closed,PDC_E,1,NaN,48,450000,275000.0,...,1568,21946,61.11,M,49.0,35833.33,RAISEN,MADHYA PRADESH,464993.0,> 48 Months
1,2,Monthly,Advance,Closed,PDC,333,BHOPAL,47,485000,350000.0,...,1062,34802,70.00,M,23.0,666.67,SEHORE,MADHYA PRADESH,466001.0,No Top-up Service
2,3,Quatrly,Arrear,Active,Direct Debit,1,NaN,68,690000,519728.0,...,1060,127335,69.77,M,39.0,45257.00,BHOPAL,MADHYA PRADESH,462030.0,12-18 Months
3,7,Monthly,Advance,Closed,Billed,125,GUNA,48,480000,400000.0,...,1060,25094,80.92,M,24.0,20833.33,ASHOK NAGAR,MADHYA PRADESH,473335.0,> 48 Months
4,8,Monthly,Arrear,Closed,Billed,152,BILASPUR,44,619265,440000.0,...,1046,21853,71.05,M,56.0,27313.67,BILASPUR,CHATTISGARH,495442.0,36-48 Months


In [122]:
#df.columns

Index(['ID', 'Frequency', 'InstlmentMode', 'LoanStatus', 'PaymentMode',
       'BranchID', 'Tenure', 'AssetCost', 'AmountFinance', 'DisbursalAmount',
       'EMI', 'AssetID', 'ManufacturerID', 'SupplierID', 'LTV', 'SEX', 'AGE',
       'MonthlyIncome', 'State', 'Top-up Month'],
      dtype='object')

In [123]:
final_df = pd.merge(df, df3, how="right", on=["ID"])

In [124]:
final_df.isnull().sum()

ID                           0
Frequency                    0
InstlmentMode                0
LoanStatus                   0
PaymentMode                  0
BranchID                     0
Tenure                       0
AssetCost                    0
AmountFinance                0
DisbursalAmount              0
EMI                          0
AssetID                      0
ManufacturerID               0
SupplierID                   0
LTV                          0
SEX                          0
AGE                          0
MonthlyIncome                0
State                        0
Top-up Month                 0
SELF-INDICATOR               0
MATCH-TYPE                   0
ACCT-TYPE                    0
CONTRIBUTOR-TYPE             0
OWNERSHIP-IND                0
ACCOUNT-STATUS               0
DISBURSED-AMT/HIGH CREDIT    0
INSTALLMENT-AMT              0
CURRENT-BAL                  0
INSTALLMENT-FREQUENCY        0
OVERDUE-AMT                  0
WRITE-OFF-AMT                0
ASSET_CL

In [125]:
final_df.columns

Index(['ID', 'Frequency', 'InstlmentMode', 'LoanStatus', 'PaymentMode',
       'BranchID', 'Tenure', 'AssetCost', 'AmountFinance', 'DisbursalAmount',
       'EMI', 'AssetID', 'ManufacturerID', 'SupplierID', 'LTV', 'SEX', 'AGE',
       'MonthlyIncome', 'State', 'Top-up Month', 'SELF-INDICATOR',
       'MATCH-TYPE', 'ACCT-TYPE', 'CONTRIBUTOR-TYPE', 'OWNERSHIP-IND',
       'ACCOUNT-STATUS', 'DISBURSED-AMT/HIGH CREDIT', 'INSTALLMENT-AMT',
       'CURRENT-BAL', 'INSTALLMENT-FREQUENCY', 'OVERDUE-AMT', 'WRITE-OFF-AMT',
       'ASSET_CLASS', 'CUR BAL - HIST', 'AMT OVERDUE - HIST',
       'AMT PAID - HIST', 'TENURE'],
      dtype='object')

In [126]:
final_df=final_df.drop(['PaymentMode','BranchID','MATCH-TYPE'],axis=1)

In [127]:
#final_df.head(5)

,ID,Frequency,InstlmentMode,LoanStatus,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,AssetID,...,INSTALLMENT-AMT,CURRENT-BAL,INSTALLMENT-FREQUENCY,OVERDUE-AMT,WRITE-OFF-AMT,ASSET_CLASS,CUR BAL - HIST,AMT OVERDUE - HIST,AMT PAID - HIST,TENURE
0,1,Monthly,Arrear,Closed,48,450000,275000.0,275000.0,24000.0,4022465,...,60961.000948,37873,F03,37873,0.0,Standard,16897.666667,12624.333333,0.0,28.801029
1,1,Monthly,Arrear,Closed,48,450000,275000.0,275000.0,24000.0,4022465,...,1405.000000,20797,F03,0,0.0,Standard,31378.347826,63.130435,0.0,36.000000
2,1,Monthly,Arrear,Closed,48,450000,275000.0,275000.0,24000.0,4022465,...,60961.000948,116087,F03,0,0.0,Standard,116024.857143,0.000000,0.0,28.801029
3,1,Monthly,Arrear,Closed,48,450000,275000.0,275000.0,24000.0,4022465,...,60961.000948,0,F03,0,0.0,Standard,118875.972973,471.916667,0.0,28.801029
4,1,Monthly,Arrear,Closed,48,450000,275000.0,275000.0,24000.0,4022465,...,60961.000948,0,F03,0,0.0,Standard,114983.000000,0.000000,0.0,28.801029


In [22]:
#final_df.columns

Index(['ID', 'Frequency', 'InstlmentMode', 'LoanStatus', 'Tenure', 'AssetCost',
       'AmountFinance', 'DisbursalAmount', 'EMI', 'AssetID', 'ManufacturerID',
       'SupplierID', 'LTV', 'SEX', 'AGE', 'MonthlyIncome', 'State',
       'Top-up Month', 'SELF-INDICATOR', 'ACCT-TYPE', 'CONTRIBUTOR-TYPE',
       'OWNERSHIP-IND', 'ACCOUNT-STATUS', 'DISBURSED-AMT/HIGH CREDIT',
       'INSTALLMENT-AMT', 'CURRENT-BAL', 'INSTALLMENT-FREQUENCY',
       'OVERDUE-AMT', 'WRITE-OFF-AMT', 'ASSET_CLASS', 'CUR BAL - HIST',
       'AMT OVERDUE - HIST', 'AMT PAID - HIST', 'TENURE'],
      dtype='object')

In [128]:
x_train=final_df.drop(['Top-up Month'],axis=1)

In [53]:
#x_train

,ID,Frequency,InstlmentMode,LoanStatus,Area,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,...,INSTALLMENT-AMT,CURRENT-BAL,INSTALLMENT-FREQUENCY,OVERDUE-AMT,WRITE-OFF-AMT,ASSET_CLASS,CUR BAL - HIST,AMT OVERDUE - HIST,AMT PAID - HIST,TENURE
0,1,Monthly,Arrear,Closed,NaN,48,450000,275000.0,275000.0,24000.0,...,60961.000948,37873,F03,37873,0.0,Standard,16897.666667,12624.333333,0.0,28.801029
1,1,Monthly,Arrear,Closed,NaN,48,450000,275000.0,275000.0,24000.0,...,1405.000000,20797,F03,0,0.0,Standard,31378.347826,63.130435,0.0,36.000000
2,1,Monthly,Arrear,Closed,NaN,48,450000,275000.0,275000.0,24000.0,...,60961.000948,116087,F03,0,0.0,Standard,116024.857143,0.000000,0.0,28.801029
3,1,Monthly,Arrear,Closed,NaN,48,450000,275000.0,275000.0,24000.0,...,60961.000948,0,F03,0,0.0,Standard,118875.972973,471.916667,0.0,28.801029
4,1,Monthly,Arrear,Closed,NaN,48,450000,275000.0,275000.0,24000.0,...,60961.000948,0,F03,0,0.0,Standard,114983.000000,0.000000,0.0,28.801029
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560839,143393,Monthly,Arrear,Active,PANIPAT,23,545000,300733.0,300733.0,15277.0,...,60961.000948,0,F03,0,0.0,Standard,94934.307692,5730.000000,0.0,12.000000
560840,143393,Monthly,Arrear,Active,PANIPAT,23,545000,300733.0,300733.0,15277.0,...,60961.000948,0,F03,0,0.0,Standard,111846.740741,23.148148,0.0,24.000000
560841,143393,Monthly,Arrear,Active,PANIPAT,23,545000,300733.0,300733.0,15277.0,...,60961.000948,11687,F03,0,0.0,Standard,232844.351351,1759.486486,0.0,48.000000
560842,143394,Half Yearly,Arrear,Active,PANIPAT,35,350000,250962.0,250962.0,74341.0,...,60961.000948,132487,F03,0,0.0,Standard,186615.466667,0.000000,0.0,28.801029


In [129]:
x_train=pd.get_dummies(x_train,columns=['Frequency','InstlmentMode','INSTALLMENT-FREQUENCY','CONTRIBUTOR-TYPE','OWNERSHIP-IND','ACCOUNT-STATUS','ACCT-TYPE','ASSET_CLASS','LoanStatus','SEX','State'])

In [130]:
#df.dtypes

ID                   int64
Frequency           object
InstlmentMode       object
LoanStatus          object
PaymentMode         object
BranchID             int64
Tenure               int64
AssetCost            int64
AmountFinance      float64
DisbursalAmount    float64
EMI                float64
AssetID              int64
ManufacturerID       int64
SupplierID           int64
LTV                float64
SEX                 object
AGE                float64
MonthlyIncome      float64
State               object
Top-up Month        object
dtype: object

In [131]:
#x_train.dtypes

ID                       int64
Tenure                   int64
AssetCost                int64
AmountFinance          float64
DisbursalAmount        float64
                        ...   
State_TAMIL NADU         uint8
State_TELANGANA          uint8
State_UTTAR PRADESH      uint8
State_UTTARAKHAND        uint8
State_WEST BENGAL        uint8
Length: 149, dtype: object

In [132]:
y_train=final_df['Top-up Month']

In [133]:
#y_train

0               > 48 Months
1               > 48 Months
2               > 48 Months
3               > 48 Months
4               > 48 Months
                ...        
560839    No Top-up Service
560840    No Top-up Service
560841    No Top-up Service
560842    No Top-up Service
560843    No Top-up Service
Name: Top-up Month, Length: 560844, dtype: object

In [134]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators = 500, min_samples_leaf = 10, max_features=0.4, max_depth=20, random_state = 123)

In [135]:
model.fit(x_train,y_train)

RandomForestClassifier(max_depth=20, max_features=0.4, min_samples_leaf=10,
                       n_estimators=500, random_state=123)

In [ ]:
###Test DAta

In [ ]:
filePath="/Users/a0p07yx/Documents/Projects/HACKATHONS/DATA/1LTFS/Test/test_data.xlsx"
tf=pd.read_excel(filePath,header=0,skiprows=0,delimiter=",")
rows,columns=tf.shape
#tf.head()

In [ ]:
filePath="/Users/a0p07yx/Documents/Projects/HACKATHONS/DATA/1LTFS/Test/test_bureau.xlsx"
tf3=pd.read_excel(filePath,header=0,skiprows=0,delimiter=",")
rows,columns=tf3.shape
#tf3.head()

In [ ]:
tf3['CUR BAL - HIST'] = tf3['CUR BAL - HIST'].apply(split_aggregate)
tf3['AMT OVERDUE - HIST'] = tf3['AMT OVERDUE - HIST'].apply(split_aggregate)
tf3['AMT PAID - HIST'] = tf3['AMT OVERDUE - HIST'].apply(split_aggregate)
tf3=tf3.drop(['CREDIT-LIMIT/SANC AMT','DATE-REPORTED','DPD - HIST','REPORTED DATE - HIST','DISBURSED-DT','CLOSE-DT','LAST-PAYMENT-DATE'],axis=1)
tf3['INSTALLMENT-AMT']=tf3['INSTALLMENT-AMT'].apply(installment_adj)
tf3['CURRENT-BAL']= tf3['CURRENT-BAL'].apply(converToInt)
tf3['DISBURSED-AMT/HIGH CREDIT']= tf3['DISBURSED-AMT/HIGH CREDIT'].apply(converToInt)
tf3['OVERDUE-AMT']= tf3['OVERDUE-AMT'].apply(converToInt)

In [ ]:
tf3['WRITE-OFF-AMT']=tf3['WRITE-OFF-AMT'].fillna(0)
tf3['OVERDUE-AMT']=tf3['OVERDUE-AMT'].fillna(0)
tf3['TENURE']=tf3['TENURE'].fillna(tf3['TENURE'].mean())
tf3['ASSET_CLASS']=tf3['ASSET_CLASS'].fillna('Standard')
tf3['CURRENT-BAL']=tf3['CURRENT-BAL'].fillna(tf3['CURRENT-BAL'].mean())
tf3['INSTALLMENT-AMT']=tf3['INSTALLMENT-AMT'].fillna(tf3['INSTALLMENT-AMT'].mean())
tf3['INSTALLMENT-FREQUENCY']=tf3['INSTALLMENT-FREQUENCY'].fillna(tf3['INSTALLMENT-FREQUENCY'].mode().iloc[0])

In [ ]:
#test_final.isnull().sum()

In [ ]:
tf=tf.drop(['DisbursalDate','MaturityDAte','AuthDate','Area','City','ZiPCODE'],axis=1)

In [ ]:
tf['SEX']=tf['SEX'].fillna(tf['SEX'].mode().iloc[0])
tf['AGE']=tf['AGE'].fillna(tf['AGE'].mode().iloc[0])
tf['MonthlyIncome']=tf['MonthlyIncome'].fillna(tf['MonthlyIncome'].mean())
tf['ManufacturerID']=tf['ManufacturerID'].fillna(tf['ManufacturerID'].mode().iloc[0])

In [ ]:
#tf.head()

In [ ]:
#tf3.head()

In [ ]:
test_final = pd.merge(tf, tf3, how="right", on=["ID"])

In [ ]:
test_final=test_final.drop(['PaymentMode','BranchID','MATCH-TYPE'],axis=1)

In [ ]:
x_test=pd.get_dummies(test_final,columns=['Frequency','InstlmentMode','INSTALLMENT-FREQUENCY','CONTRIBUTOR-TYPE','OWNERSHIP-IND','ACCOUNT-STATUS','ACCT-TYPE','ASSET_CLASS','LoanStatus','SEX','State'])

In [ ]:
# Get missing columns in the training test
missing_cols = set( x_train.columns ) - set( x_test.columns )
# Add a missing column in test set with default value equal to 0
for c in missing_cols:
    x_test[c] = 0
# Ensure the order of column in the test set is in the same order than in train set
x_test = x_test[x_train.columns]

In [ ]:
prediction=model.predict(x_test)

In [ ]:
result=pd.DataFrame()
result['ID']=x_test['ID']

In [ ]:
result.loc[:,'Top-up Month']=prediction
#result

In [ ]:
rf2=result.groupby('ID')['Top-up Month'].apply(lambda x: x.first()).reset_index()

In [ ]:
rf2=rf2.reset_index()

In [ ]:
rf2.to_csv('result5.csv', index=False)